# Autoregressive text generation with Llama3

Language models are (nowadays) used to predict a likely continuation of a given context.

The model estimates a probability density function over all possible next elements in the sequence. We then select the best-scored element or perform a weighted sampling from the probability density function. 

After selecting the next element, we act as if this were *correct*, i.e., we predict the elements afterwards by adding the already selected elements to the context.

# Tokenization

Text data is sequential, i.e. a sequence of words following each other, or a sequence of characters. The nature of the elements that make up the sequence is important, as these elements (called *tokens*) determine how the LM "sees" texts.

In the past, LMs used words as tokens (e.g. in speech recognition), or characters (e.g. in spelling correction). A disadvantage of words-as-tokens is that we can never include **all** words (language users come up with new words all the time). A disadvantage of characters-as-tokens is that they do not have any **meaning** (which is important to determine continuations in most cases) and that meaning only emerges from variably sized sequences of characters. Furthermore, the a priori probability of characters is very imbalanced ('e' is much more probable than 'q' in most languages), which impedes model performance. Also, there are only very few distinct characters, far fewer than our models can deal with nowadays.

Nowadays, we use as tokens short character sequences that:
* completely cover all possible character sequences (by including all individual characters)
* contain frequent words as individual tokens
* most often **make sense**, when words need to be broken into multiple tokens.
* never span multiple words and contain whitespace only at the beginning (these are artifacts of the token generation procedure).

Some examples: 
* <tt><span style="background:lightsalmon">Not</span><span style="background:powderblue"> just</span><span style="background:lavender"> one</span><span style="background:lightsalmon"> rainbow</span><span style="background:powderblue"> but</span><span style="background:lavender"> two</span><span style="background:lightsalmon"> rain</span><span style="background:powderblue">bows</span><span style="background:lavender"> can</span><span style="background:lightsalmon"> be</span><span style="background:powderblue"> seen</span><span style="background:lavender">.</span></tt>
* <tt><span style="background:lightsalmon">N</span><span style="background:powderblue">icht</span><span style="background:lavender"> nur</span><span style="background:lightsalmon"> ein</span><span style="background:powderblue"> Reg</span><span style="background:lavender">en</span><span style="background:lightsalmon">b</span><span style="background:powderblue">o</span><span style="background:lavender">gen</span><span style="background:lightsalmon">,</span><span style="background:powderblue"> sondern</span><span style="background:lavender"> mehr</span><span style="background:lightsalmon">ere</span><span style="background:powderblue"> Reg</span><span style="background:lavender">en</span><span style="background:lightsalmon">b</span><span style="background:powderblue">ö</span><span style="background:lavender">gen</span><span style="background:lightsalmon"> sind</span><span style="background:powderblue"> s</span><span style="background:lavender">icht</span><span style="background:lightsalmon">bar</span><span style="background:powderblue">.</span></tt><br>
→ Note how German text contains shorter tokens -- this puts German at a disadvantage as compared to English. (This is primarily an artifact of the training material, which is mostly in English.)

To check the tokenization of some text, check out: https://belladoreai.github.io/llama3-tokenizer-js/example-demo/build/ . (Unfortunately, tokens aren't explicitly shown below.)

There are two methods for token generation from texts (or word lists) at the moment: [Byte-pair encoding (BPE)](https://en.wikipedia.org/wiki/Byte_pair_encoding) and [unigram modelling](https://huggingface.co/learn/nlp-course/en/chapter6/7). See e.g. [SentencePiece](https://github.com/google/sentencepiece) for a software that implements both.

# Playground! 

Below, you can play around to see generation in action. Please use the "Run all Cells" command (from the menu, or via the fast-forward icon) to load up the system. 

You will then see a text field for the context (that forms the base for generation) and a button that generates the most likely next token given the context. 

For example, the context <tt style="background:lavender">>The sky is</tt> yields the next token <tt style="background:lavender">&nbsp;the</tt>. The generated token is automatically added to the context, so you can click generate multiple times to see the autoregressive completion of the context.

In [ ]:
!pip3 install ollama
import ipywidgets as widgets
import termcolor
import ollama

In [ ]:
def handle_click_generate(queryText, tempSlider, responseText, button):
    print("HELLO")
    responseText.clear_output()
    temp = tempSlider.value
    with responseText:
        print("'" + termcolor.colored(queryText.value, color='blue') + "'", end='→')
        response = ollama.generate(prompt=queryText.value, model='llama3:70b-text', template='{{ .Prompt }}', options={"temperature":temp, "num_predict":1})
        print("'" + termcolor.colored(str(response['response']), color='red') + "'")
        queryText.value += response['response']

In [ ]:
# set up the widgets to be used interactively
def createChatInterface(click_handler, initial_value='The sky is'):
    queryText = widgets.Text(
        value=initial_value,
        placeholder='Type something',
        description='Context:',
        disabled=False
    )
    tempSlider = widgets.FloatSlider(
        value=0.0,
        min=0.0,
        max=10.0,
        description='Sampling temperature', 
        style={'description_width': 'initial', 'width': '800px'},
    )
    button = widgets.Button(
        description='Generate',
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='generate',
        icon='check' # (FontAwesome names without the `fa-` prefix)
    )
    responseText = widgets.Output()
    button.on_click(lambda b:click_handler(queryText, tempSlider, responseText, b))
    
    display(widgets.HBox([queryText, button, tempSlider]))
    display(responseText)
    
createChatInterface(handle_click_generate)

# More experimentation with autoregressive text continuation

* try German texts, you'll notice the shorter tokens (more clicks on "Generate")
* ***sampling temperature***: the sampling temperature influences the sampling strategy: 
   * temperature=0 always selects the most-probable token of the probability density function
   * temperatures>0 enables weighted sampling from the pdf. temperature=1 uses the original weights; the higher the value, the more '''creative''' (rather: varied) the responses.
   * temperatures near infinity yield random sampling among the token vocabulary
* try setting a high sampling temperature and repeatedly generate from the same context. You will get different results most of the time.
* What is the continuation for <tt style="background:lavender">What is the color of the sea?</tt> (try with temperature=0 and maybe also with higher temperatures; try to explain what's happening; is the continuation as expected?)

# Make models *respond* to questions:
For a language model, most likely continuations to questions are: more questions. This is an artifact of often seeing lists of questions in the training material (e.g. https://www.teenvogue.com/story/questions-to-get-to-know-someone).

A superpower of large language models is **in-context learning**: You provide the model with some insight as to what you expect the continuation to be like by providing some examples. Try: <tt style="background:lavender">What is the color of grass? green. What is the color of the sun? yellow. What is the color of the sky?</tt> The model is now able to generate answers! (But do look at the continuation beyond the response...) 

Providing examples in-context is very nice, because the model does not need to be retrained for the specific task. People speak of **one-shot** or **few-shot learning** (in our case above: 2-shot), to describe the situation that the model successfully performs a task when provided with one or a few examples. However, notice that no form of *learning* is taking place (my definition of learning includes some lasting effect). Instead, the model has previously learned to understand patterns of questions+answers and to continue them -- for all kinds of questions!

Also, compare the continuations of <tt style="background:lavender">At what time does the train leave?</tt> and <tt style="background:lavender">At what time does the airplane leave? At 6am. At what time does the train leave?</tt>. When we're asking about non-**world-level* information, we'll have to do some serious **prompt engineering**.

Today's LLMs are not strictly optimized to generate most likely sequence continuations but they are **aligned** to produce outputs that *make sense* when used as a chatbot. The most basic form of this is using markers to designate user- and system-turns in the context. Try: <tt style="background:lavender">user: What is the color of the sea? system:</tt>. Also try: <tt style="background:lavender">user: At what time does the train leave? system:</tt> Notice that this works without *any* example which is why people call it **zero-shot learning**.

# Chatbot prompts:
Above, we used the LLM in a mode that yields "only" text continuations and gradually looked at how extensions to the context are used to generate chatbot-like responses. Normally, chatbots come with a default prompt template, into which the user's input is added. For our model this is:

<pre>{{ if .Prompt }}<|start_header_id|>user<|end_header_id|>

{{ .Prompt }}<|eot_id|>{{ end }}<|start_header_id|>assistant<|end_header_id|>

{{ .Response }}<|eot_id|></pre>

(note this is very close to what we did above, except that here, model-specific special tokens are used: <tt><|start_header_id|></tt>, <tt><|eot_id|></tt>, ...

To get this behaviour, replace the button handler in the GUI code above <tt>button_click(handle_click_generate)</tt> as described below. Note that the handler below will also produce as many tokens as the model desires (until it generates the special token <tt><|eot_id|></tt>).

In [ ]:
def handle_click_chat(queryText, tempSlider, responseText, button):
    #responseText.clear_output()
    temp = tempSlider.value
    template = """
<|im_start|>system 
<|im_end|>
{{ if .Prompt }}<|start_header_id|>user<|end_header_id|>
You're a dashboard. You produce output for JSON queries. 
If the input is:
{ type:"bip", year:1992, state:"thüringen", value:"30000DM"}
the output shall be:
Thüringen hatte im Jahr 1992 ein Bruttoinlandsprodukt von 30000 DM.
If the input is:
{ type:"bws", year:2000, state:"bayern", value:"500000€"}
the output shall be:
Die Bruttowertschöpfung lag in Bayern im Jahr 2000 bei 50 TEUR.

Please do not restate the examples above. Only try to turn the input JSON into its output but do not explain too much.
{{ .Prompt }}<|eot_id|>{{ end }}<|start_header_id|>assistant<|end_header_id|>
{{ .Response }}<|eot_id|>
"""
    with responseText:
        button.disabled = True # responses will be much longer for this.
        print("'" + termcolor.colored(queryText.value, color='blue') + "'", end='→')
        response = ollama.generate(prompt=queryText.value, template=template, model='llama3.1:70b', options={"temperature":temp})
        print("'" + termcolor.colored(str(response['response']), color='red') + "'")
        # queryText.value += response['response'] # doesn't really make sense in this situation
        button.disabled = False


createChatInterface(handle_click_chat, '{ type:"einwohnerzahl", year:2024, city:"Regensburg", value:"150000"}')

# Retrieval-augmented Generation
Retrieval Augmented Generation is a two-step process: in the first step, some search mechanism some finds documents (or parts of documents) that are relevant to the question posed to the model. In the second step, the documents, some instructions and the original question are combined and passed to the LLM. 

Example prompt: 

```
Instruction: Write an accurate, engaging, and concise answer for the given question using only the provided documents (some of which might be irrelevant) and cite them properly using the format [<doc_id>]. For example if a particular information is from document 3 reference it with [3].
Documents:
[1]: Hollywood was a small community in 1870 and was incorporated as a municipality in 1903. It was consolidated with the city of Los Angeles in 1910, and soon thereafter a prominent film industry emerged, eventually becoming the most recognizable film industry in the world.
[2]: By 1912, major motion-picture companies had set up production near or in Los [...]
[...]
[10]: The history of the Forest Service [...]
Question: When did hollywood become the centre of the film industry?
Model output: Hollywood, as part of Los Angeles became the centre of the film industry between 1903 [1] and 1912 [2].
```
→ see also our recent publication on evaluating and improving citation performance in RAG!

Note that the search mechanism that finds documents relevant to the user query is not part of the language model. It can be done in many ways, one of them is based on BERT-style models (finding documents that have document encodings that are very similar to the query encoding).

# LLMs with Function Calling
In the past, LLMs would, if faced with a web-query, hallucinate the response to the query. Also, math worked (and still works) incredibly badly. (It's just very likely, that the addition of say, two 20 digit numbers, will fail at least for one digit; while LLMs get by with vagueness in language quite well, a response in maths is either correct or false and the vast majority of possible responses is false.)

However, an LLM can be instructed (and fine-tuned) to generate, instead of a response, a function call when it is faced with a query that it cannot respond.

```
User Query: "What is 3 + 4?"
Model output: "INTERNAL QUERY TO math model FOR: 3 + 4" → this response is not sent back to the user but to the math model (based on the marker words found).
Math model: 3 + 4 = 7
Amended User Query (as in RAG): "Respond to the following question given that the math model has stated that 3 + 4 = 7: What is 3 + 4?"
Model output: The answer to your question is 7.
```

Note that the math model (and many other models) is not part of the language model (for math, a calculator would be useful, for other tasks, it could check online databases, etc.) and neither is the mechanism for filtering out the "INTERNAL QUERYs", calling the external model and feeding back the model response. The LM is merely trained to generate these internal queries when appropriate.

While the internal models need not necessarily be LMs, one could of course use LMs that are fine-tuned for specific purposes. This is called agent-based LLM-processing. 

### LLMs have no clue what they are doing

For example, I wanted to get a word that consists of multiple tokens. 

<tt>give me a word that internally consists of multiple tokens.</tt> → <tt>How about "toothbrush"? It can be broken down into three internal tokens: "tooth" and "brush".</tt> However, according to https://belladoreai.github.io/llama3-tokenizer-js/example-demo/build/ it consists of three tokens: <tt>to-oth-brush</tt> :-(

Likewise, it also told me about bookshelf (with a correct break-down). I mistyped it in the tokenizer (<tt>book-sh-elp</tt>), so I asked the breakdown for that (mistyped) word: <tt>how many tokens is bookshelp</tt> → <tt>The word "bookshelp" can be broken down into two internal tokens: "book" and "shelp".</tt>. I didn't give up: <tt>try again</tt> → <tt>I apologize for the mistake! The correct answer is that "bookshelp" can be broken down into two internal tokens: "books" and "help".</tt>. It went on to claim that "bookshelp" is just one token. It never got the correct tokenization, although it had just seen it in the input.

Bottom line: the model is made to predict next tokens; that's what it's good at. It is not made to analyze its input tokens and often fails miserably if asked to do that.
